<a href="https://colab.research.google.com/github/balakumar-dataengineer/testrepo/blob/master/Pyspark_Window_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('pyspark') \
                          .master('local') \
                          .getOrCreate()

print(spark)

In [5]:
data=[
    (1, "Alice", "HR", 63000),
    (2, "Bob", "HR", 52000),
    (4, "David", "HR", 55000),
    (12, "Leo", "Finance", 63000),
    (13, "Mia", "Finance", 63000),
    (5, "Eve", "HR", 59000),
    (6, "Frank", "IT", 89000),
    (7, "Grace", "IT", 72000),
    (3, "Charlie", "HR", 53000),
    (8, "Hank", "IT", 73000),
    (9, "Ivy", "IT", 73000),
    (10, "Jack", "IT", 76000),
    (11, "Kate", "Finance", 60000),
    (14, "Nina", "Finance", 90000),
    (15, "Oscar", "Finance", 69000)
]

schema=['id','name','dep','salary']

df=spark.createDataFrame(data,schema)
df.show()
df.printSchema()

+---+-------+-------+------+
| id|   name|    dep|salary|
+---+-------+-------+------+
|  1|  Alice|     HR| 63000|
|  2|    Bob|     HR| 52000|
|  4|  David|     HR| 55000|
| 12|    Leo|Finance| 63000|
| 13|    Mia|Finance| 63000|
|  5|    Eve|     HR| 59000|
|  6|  Frank|     IT| 89000|
|  7|  Grace|     IT| 72000|
|  3|Charlie|     HR| 53000|
|  8|   Hank|     IT| 73000|
|  9|    Ivy|     IT| 73000|
| 10|   Jack|     IT| 76000|
| 11|   Kate|Finance| 60000|
| 14|   Nina|Finance| 90000|
| 15|  Oscar|Finance| 69000|
+---+-------+-------+------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- dep: string (nullable = true)
 |-- salary: long (nullable = true)



In [26]:
from pyspark.sql.functions import rank,dense_rank,row_number,sum,col,max,count
from pyspark.sql.window import Window

window=Window.partitionBy('dep').orderBy('salary')

df.withColumn('rownumber',row_number().over(window)) \
  .withColumn('rank',rank().over(window)) \
  .withColumn('dense_rank',dense_rank().over(window)) \
  .show()

+---+-------+-------+------+---------+----+----------+
| id|   name|    dep|salary|rownumber|rank|dense_rank|
+---+-------+-------+------+---------+----+----------+
| 11|   Kate|Finance| 60000|        1|   1|         1|
| 12|    Leo|Finance| 63000|        2|   2|         2|
| 13|    Mia|Finance| 63000|        3|   2|         2|
| 15|  Oscar|Finance| 69000|        4|   4|         3|
| 14|   Nina|Finance| 90000|        5|   5|         4|
|  2|    Bob|     HR| 52000|        1|   1|         1|
|  3|Charlie|     HR| 53000|        2|   2|         2|
|  4|  David|     HR| 55000|        3|   3|         3|
|  5|    Eve|     HR| 59000|        4|   4|         4|
|  1|  Alice|     HR| 63000|        5|   5|         5|
|  7|  Grace|     IT| 72000|        1|   1|         1|
|  8|   Hank|     IT| 73000|        2|   2|         2|
|  9|    Ivy|     IT| 73000|        3|   2|         2|
| 10|   Jack|     IT| 76000|        4|   4|         3|
|  6|  Frank|     IT| 89000|        5|   5|         4|
+---+-----

In [28]:
window1=Window.partitionBy('dep').orderBy(col('salary').asc(),col('id').desc())

df.withColumn('cum_salary',sum(df.salary).over(window1)).show()

+---+-------+-------+------+----------+
| id|   name|    dep|salary|cum_salary|
+---+-------+-------+------+----------+
| 11|   Kate|Finance| 60000|     60000|
| 13|    Mia|Finance| 63000|    123000|
| 12|    Leo|Finance| 63000|    186000|
| 15|  Oscar|Finance| 69000|    255000|
| 14|   Nina|Finance| 90000|    345000|
|  2|    Bob|     HR| 52000|     52000|
|  3|Charlie|     HR| 53000|    105000|
|  4|  David|     HR| 55000|    160000|
|  5|    Eve|     HR| 59000|    219000|
|  1|  Alice|     HR| 63000|    282000|
|  7|  Grace|     IT| 72000|     72000|
|  9|    Ivy|     IT| 73000|    145000|
|  8|   Hank|     IT| 73000|    218000|
| 10|   Jack|     IT| 76000|    294000|
|  6|  Frank|     IT| 89000|    383000|
+---+-------+-------+------+----------+



In [34]:
df.withColumn('rank',rank().over(window)).filter(col('rank')==5).drop('rank').show()

+---+-----+-------+------+
| id| name|    dep|salary|
+---+-----+-------+------+
| 14| Nina|Finance| 90000|
|  1|Alice|     HR| 63000|
|  6|Frank|     IT| 89000|
+---+-----+-------+------+

